In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
new_data = "./DPIC-post-2002.csv" 
new_data_df = pd.read_csv(new_data) 

In [3]:
old_data="./ICPSR-pre-2002.csv"
old_data_df =pd.read_csv(old_data)

In [4]:
old_data_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
0,8451,4,1,10001,1,,NORMAN ELI,2,4,1,...,19,12,1812,1,1,1089,41,1,,250
1,8451,4,1,10002,1,,WOODS JOHN,4,6,44,...,6,2,1814,1,1,1000,41,1,,616
2,8451,4,1,10003,2,,GAMARRA CHARLES,2,2,13,...,19,5,1820,1,1,1097,41,1,,
3,8451,4,1,10004,2,,DESAYAS VICTORIANO,2,2,1,...,19,5,1820,2,1,1097,41,1,,
4,8451,4,1,10005,2,,NOREAGO FRANCISCO,2,2,1,...,30,5,1822,1,1,1097,41,1,,187


In [5]:
column_decode = {'V5':'Race','V7':'Name','V10':'Crimes Committed','V11':'Execution Method','V12':'Day','V13':'Month','V14':'Year','V16':'State','V19':'Sex'}
column_encode=dict(zip(column_decode.values(),column_decode.keys()))

In [6]:
old_data_df=old_data_df.rename(mapper=column_decode,axis='columns')

In [7]:
old_select=old_data_df[['Race','Name','Execution Method','State','Sex','Year','Month','Day','Crimes Committed']]

In [8]:
new_data_df.head()

,Execution#,Execution Date,First Name,Last Name,Middle Name(s),Suffix,Race,Sex,Region,State,...,Number of Latino Male Victims,Number of Asian Male Victims,Number of Native American Male Victims,Number of Other Race Male Victims,Number of White Female Victims,Number of Black Female Victims,Number of Latino Female Victims,Number of Asian Female Victims,Number of Native American Female Victims,Number of Other Race Female Victims
0,1,01/17/1977,Gary,Gilmore,Mark,NaN,White,Male,West,Utah,...,0,0,0,0,0,0,0,0,0,0
1,2,05/25/1979,John,Spenkelink,NaN,NaN,White,Male,South,Florida,...,0,0,0,0,0,0,0,0,0,0
2,3,10/22/1979,Jesse,Bishop,NaN,NaN,White,Male,West,Nevada,...,0,0,0,0,0,0,0,0,0,0
3,4,03/09/1981,Steven,Judy,NaN,NaN,White,Male,Midwest,Indiana,...,0,0,0,0,2,0,0,0,0,0
4,5,08/10/1982,Frank,Coppola,NaN,NaN,White,Male,South,Virginia,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#split dates into 3 columns
all_dates = new_data_df['Execution Date']
all_split = [date.split('/') for date in all_dates]
month=[arr[0] for arr in all_split]
day=[arr[1]for arr in all_split]
year = [arr[2] for arr in all_split]
new_data_df['Month']=month
new_data_df['Day']=day
new_data_df['Year']=year

In [10]:
#combine first and last into 1 column
new_data_df['Name']=new_data_df['First Name'].map(str)+' '+new_data_df['Last Name']
new_data_df['Name']=[name.upper() for name in new_data_df['Name']]
#add crimes committed column
new_data_df['Crimes Committed']=1

In [11]:
new_select=new_data_df[['Race','Name','Execution Method','State','Sex','Year','Month','Day','Crimes Committed']]

In [12]:
new_select.head()

,Race,Name,Execution Method,State,Sex,Year,Month,Day,Crimes Committed
0,White,GARY GILMORE,Firing Squad,Utah,Male,1977,01,17,1
1,White,JOHN SPENKELINK,Electrocution,Florida,Male,1979,05,25,1
2,White,JESSE BISHOP,Gas,Nevada,Male,1979,10,22,1
3,White,STEVEN JUDY,Electrocution,Indiana,Male,1981,03,09,1
4,White,FRANK COPPOLA,Electrocution,Virginia,Male,1982,08,10,1


In [ ]:
#old_select=old_select.drop(['Crimes Committed'],axis=1)

In [13]:
#join old and new
all_records=pd.concat([new_select,old_select])
all_records.size

150966

In [14]:
#drop duplicate names
deduped_all = all_records.drop_duplicates(subset='Name',keep='first')
deduped_all.size

136323

In [15]:
all_records=deduped_all.drop(['Name'],axis=1)
all_records=all_records.dropna()
all_records.size

121176

# Make decoded & encoded dfs

In [16]:
def decode(input_df, decode_dict,column):
    return input_df[column].replace(decode_dict.keys(),value=decode_dict.values())
def encode(input_df, decode_dict,column):
    return input_df[column].replace(decode_dict.values(),value=decode_dict.keys())

In [33]:
state_key= "./ICPSR_08451/state-fips.csv"
state_df=pd.read_csv(state_key)
code=state_df[' st']
state=state_df['stname']
state_decode=dict(zip(code,state))
state_decode

{1: 'Alabama',
 2: 'Alaska',
 4: 'Arizona',
 5: 'Arkansas',
 6: 'California',
 8: 'Colorado',
 9: 'Connecticut',
 10: 'Delaware',
 11: 'District of Columbia',
 12: 'Florida',
 13: 'Georgia',
 15: 'Hawaii',
 16: 'Idaho',
 17: 'Illinois',
 18: 'Indiana',
 19: 'Iowa',
 20: 'Kansas',
 21: 'Kentucky',
 22: 'Louisiana',
 23: 'Maine',
 24: 'Maryland',
 25: 'Massachusetts',
 26: 'Michigan',
 27: 'Minnesota',
 28: 'Mississippi',
 29: 'Missouri',
 30: 'Montana',
 31: 'Nebraska',
 32: 'Nevada',
 33: 'New Hampshire',
 34: 'New Jersey',
 35: 'New Mexico',
 36: 'New York',
 37: 'North Carolina',
 38: 'North Dakota',
 39: 'Ohio',
 40: 'Oklahoma',
 41: 'Oregon',
 42: 'Pennsylvania',
 44: 'Rhode Island',
 45: 'South Carolina',
 46: 'South Dakota',
 47: 'Tennessee',
 48: 'Texas',
 49: 'Utah',
 50: 'Vermont',
 51: 'Virginia',
 53: 'Washington',
 54: 'West Virginia',
 55: 'Wisconsin',
 56: 'Wyoming'}

In [74]:
race_decode={
"0M": "DoNotKnow;NA", 
"1" : "White",
"2" : "Black",
"3" : "Native American",
"4" : "Asian",
"5" : "Latino",
"6" : "Other Race"}
sex_decode={'0M':'DoNotKnow;NA','1': 'Male',
'2': 'Female'}
EM_decode={"0M": "DoNotKnow;NA",
"1": "Hanging",
"2": "Electrocution",
"3" : "Gas",
"4" :"Firing Squad",
"5": "Lethal Injection",
"6": "Pressing",
"8": "Break on Wheel",
"10": "Burned",
"11": "Hung in Chains",
"13" : "Bludgeoned",
"14" :  "Gibbetted",
"15" :  "Other"}

In [80]:
#everything is decoded except for crimes committed
all_decoded=all_records
all_decoded['Race']=decode(all_decoded,race_decode,'Race')
all_decoded['Sex']=decode(all_decoded,sex_decode,'Sex')
all_decoded['Execution Method']=decode(all_decoded,EM_decode,'Execution Method')
all_decoded['State']=decode(all_decoded,state_decode,'State')
#detect cells with white space as na, drop them
all_decoded=all_decoded.replace(r'^\s*$', np.nan, regex=True).dropna()
all_decoded.size

110224

In [84]:
all_encoded=all_records
all_encoded['Race']=encode(all_encoded,race_decode,'Race')
all_encoded['Sex']=encode(all_encoded,sex_decode,'Sex')
all_encoded['Execution Method']=encode(all_encoded,EM_decode,'Execution Method')
all_encoded['State']=encode(all_encoded,state_decode,'State')
all_encoded=all_encoded.replace(r'^\s*$', np.nan, regex=True).dropna()
all_encoded.size

110224

In [81]:
all_encoded.to_csv('./all_encoded.csv')
all_decoded.to_csv('./all_decoded.csv')

In [82]:
all_encoded['Year']=pd.to_numeric(all_encoded['Year'],errors='coerce')
pre_1974_en=all_encoded.loc[all_encoded['Year'] < 1974]
post_1974_en=all_encoded.loc[all_encoded['Year']>= 1974]
all_decoded['Year']=pd.to_numeric(all_decoded['Year'],errors='coerce')
pre_1974_de=all_decoded.loc[all_decoded['Year'] < 1974]
post_1974_de=all_decoded.loc[all_decoded['Year'] >= 1974]

In [83]:
pre_1974_en.to_csv('./pre_1974_encoded.csv')
pre_1974_de.to_csv('./pre_1974_decoded.csv')
post_1974_en.to_csv('./post_1974_encoded.csv')
post_1974_de.to_csv('./post_1974_decoded.csv')